In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
NivelMax = ([3, 1, 2])

In [3]:
hierarquias = [NivelMax]

In [4]:
def preProcessamento(df):
    df = doencas.copy() 
    local = []
    cidades = doencas.cidade
    estados = doencas.estado

    for i in range(0,len(cidades)):
        local.append(str('{}, {}'.format(cidades[i], estados[i])))
        
    pd.DataFrame(local)
    df = df.drop("cidade",axis=1)
    df = df.drop("estado",axis=1)

    df['local'] = local

    sequence = ['id', 'data', 'genero', 'local', 'doenca']
    df = df.reindex(columns=sequence)
    
    return df

In [5]:
def preProcessamentoBack(df):
    df = background.copy() 
    local = []
    cidades = background.cidade
    estados = background.estado

    for i in range(0,len(cidades)):
        local.append(str('{}, {}'.format(cidades[i], estados[i])))
        
    pd.DataFrame(local)
    df = df.drop("cidade",axis=1)
    df = df.drop("estado",axis=1)

    df['local'] = local

    sequence = ['nome', 'data', 'genero', 'local']
    df = df.reindex(columns=sequence)
    
    return df

In [6]:
def gerarHierarquias(NivelMax):
    hierarquias = [NivelMax]
    while([0,0,0] not in hierarquias):
        for i in range(0, len(hierarquias)):
            for j in range(0, len(NivelMax)):
                h = hierarquias[i].copy()
                h[j] = h[j] - 1
                if(h[j]>=0):
                    if(h not in hierarquias):
                        hierarquias.append(h)
    return hierarquias

In [7]:
def buscaNivel(elemento):
    return elemento.count('*')

In [8]:
def precisao(df, N, D):
    somatorio = 0
    for i in range(0,N):
        for j in range(0,D):
            if(i == 0):                
                h = buscaNivel(df[j][i])
                HGV = 3
            elif(i == 1):
                h = buscaNivel(df[j][i])
                HGV = 1
            else:
                h = buscaNivel(df[j][i])
                HGV = 2
            somatorio += h/HGV
    resultado = 1 - (somatorio)/(D*N)
    return resultado

In [9]:
def verificarDelta(dataset, datasetBackground, delta):
    regDataset = []
    contRegDataset = []
    regBack = []
    contRegBack = []
    for i in dataset:
        reg = [i[1],i[2], i[3]]
        cont = 1
        if(reg not in regDataset):
            regDataset.append(reg)
            contRegDataset.append(cont)
        else:
            for j in range(0, len(regDataset)):
                if(reg == regDataset[j]):
                    contRegDataset[j] = contRegDataset[j] + 1
    for i in datasetBackground:
        reg = [i[1],i[2], i[3]]
        cont = 1
        if(reg not in regBack):
            regBack.append(reg)
            contRegBack.append(cont)
        else:
            for j in range(0, len(regBack)):
                if(reg == regBack[j]):
                    contRegBack[j] = contRegBack[j] + 1
    for i in range(0, len(regBack)):
        if(regDataset[0] == regBack[i]):
            probMin = contRegDataset[0]/contRegBack[i]
            probMax = contRegDataset[0]/contRegBack[i]
            
    for i in range(0, len(contRegDataset)):
        for j in range(0, len(contRegBack)):
            if(regDataset[i] == regBack[j]):
                prob = contRegDataset[i]/contRegBack[j]
                if(prob <= probMin):
                    probMin = prob
                elif(prob > probMax):
                    probMax = prob
    if(probMin>=delta[0] and probMax<=delta[1]):
        return True
    else:
        return False

In [10]:
def verificarPrecisao(datasets):
    precisoes = []
    for dataset in datasets:
        precisoes.append(precisao(dataset, 3, len(datasets[0])))
    precisoes = np.array(precisoes)
    return datasets[np.argmax(precisoes)]

In [ ]:
def posProcessamento(df):
    

In [11]:
def deltaPresenca(df, datasetBackground, delta, hierarquias):
    df = np.array(df)
    dfBack = np.array(datasetBackground)
    datasets = []
    for h in hierarquias:
        dado = np.copy(df)
        dadoBack = np.copy(dfBack)
        for i in dado:
            data = np.array(i[1].split("/"))
            local = np.array(i[3].split(","))
            if(h[0] == 3):
                i[1] = '*/*/*'
            elif(h[0] == 2):
                i[1] = '*/*/'+data[2]
            elif(h[0] == 1):
                i[1] = '*/'+data[1]+'/'+data[2]
            if(h[1] == 1):
                i[2] = "*"
            if(h[2] == 2):
                i[3] = "*,*"
            elif(h[2] == 1):
                i[3] = "*,"+local[1]
        for i in dadoBack:
            data = np.array(i[1].split("/"))
            local = np.array(i[3].split(","))
            if(h[0] == 3):
                i[1] = '*/*/*'
            elif(h[0] == 2):
                i[1] = '*/*/'+data[2]
            elif(h[0] == 1):
                i[1] = '*/'+data[1]+'/'+data[2]
            if(h[1] == 1):
                i[2] = "*"
            if(h[2] == 2):
                i[3] = "*,*"
            elif(h[2] == 1):
                i[3] = "*,"+local[1]
        dadoValido = verificarDelta(dado, dadoBack, delta)
        if(dadoValido):
            dataset = pd.DataFrame(dado, columns=['id','data','genero','local','doenca'])
            dataset = dataset.drop('id', axis=1)
            dataset = np.array(dataset)
            datasets.append(dataset)
    datasetFinal = verificarPrecisao(datasets)
    
    return datasetFinal

In [12]:
doencas = pd.read_csv('doencas.csv')
background = pd.read_csv('background.csv')

In [13]:
dfDoencas = preProcessamento(doencas)
dfBackground = preProcessamentoBack(background)

In [14]:
dfBackground

nome        data  genero                      local
0     Alessandro Cowterd  27/12/1999    Male    New York City, New York
1       Cathie Bernhardi  07/10/1999  Female         Brooklyn, New York
2     Dorothee McCluskey  14/12/1999  Female          Buffalo, New York
3            Mace Scrine  13/03/2000    Male    New York City, New York
4           Ethyl Suttie  18/11/1999  Female    New York City, New York
5      Zebulon McSperron  23/11/1999    Male         Brooklyn, New York
6       Christyna Bredes  18/12/1999  Female    Shawnee Mission, Kansas
7           Marlin Brent  17/10/1999    Male      Schenectady, New York
8         Lindsay Gerbel  08/10/1999    Male       Great Neck, New York
9         Corena Jennins  02/02/2000  Female            Bronx, New York
10            Nona Moule  23/11/1999  Female            Wichita, Kansas
11        Doralyn Elcome  22/10/1999  Female    New York City, New York
12      Danit Feldharker  21/02/2000  Female    New York City, New York
13      Juliann Hindshaw  13/02/2000  Female        Rochester, New York
14       Vassily Barnett  31/10/1999    Male    New York City, New York
15         Alicea Camsey  10/02/2000  Female    Staten Island, New York
16         Jamie Frotton  25/02/2000  Female       Hicksville, New York
17        Kassie Rydings  24/02/2000  Female    Shawnee Mission, Kansas
18          Ade Blemings  21/10/1999    Male    New York City, New York
19      Lancelot Lynnett  29/02/2000    Male    New York City, New York
20        Danice Hardwin  09/12/1999  Female    Shawnee Mission, Kansas
21           Conny Skace  17/10/1999  Female           Albany, New York
22         Gregory Lowne  09/10/1999    Male        Rochester, New York
23          Elyn Jenking  12/12/1999  Female        Rochester, New York
24      Phyllys Filipyev  13/12/1999  Female         Flushing, New York
25        Carol O'Shiels  24/02/2000  Female          Jamaica, New York
26          Gusti Elcoat  06/03/2000  Female    New York City, New York
27      Hilliard Gosnold  19/11/1999    Male         Syracuse, New York
28     Juliette Whaymand  08/10/1999  Female          Buffalo, New York
29           Ellary Bird  01/10/1999    Male         Brooklyn, New York
...                  ...         ...     ...                        ...
1970  Hermie Skentelbury  12/02/2000    Male            Wichita, Kansas
1971     Anatollo Ailmer  16/12/1999    Male             Topeka, Kansas
1972        Hazel Gronou  03/12/1999  Female    New York City, New York
1973     Brenda Blampied  16/12/1999  Female      Schenectady, New York
1974     Chucho Houndsom  06/10/1999    Male         Syracuse, New York
1975       Debor Charles  14/11/1999  Female        Rochester, New York
1976      Jsandye Waight  15/11/1999  Female    New York City, New York
1977      Stefania Janic  22/10/1999  Female    New York City, New York
1978   Crystie Hollyland  05/03/2000  Female          Buffalo, New York
1979     Derek Grinnikov  15/10/1999    Male    New York City, New York
1980     Bengt Tomblings  29/01/2000    Male          Buffalo, New York
1981       Malina Winham  25/02/2000  Female         Syracuse, New York
1982         Brett Lemar  05/03/2000  Female    New York City, New York
1983     Shelley Mandell  07/12/1999    Male         Flushing, New York
1984       Nobie Hurtado  05/01/2000    Male             Topeka, Kansas
1985         Zach Headly  14/03/2000    Male    New York City, New York
1986      Karlan Ullyatt  04/10/1999    Male    New York City, New York
1987          Rob Fernie  02/02/2000    Male             Topeka, Kansas
1988      Fairlie Petett  28/12/1999    Male        Rochester, New York
1989     Cassey Stuffins  18/11/1999  Female    New York City, New York
1990    Leonie Otterwell  13/02/2000  Female             Topeka, Kansas
1991       Abraham Ragat  26/12/1999    Male  Port Washington, New York
1992    Corissa Devenish  13/11/1999  Female            Wichita, Kansas
1993  Hermann Hemmingway  06/11/1999    Male     White Plain

In [15]:
hierarquias = gerarHierarquias(NivelMax)

In [16]:
final = deltaPresenca(dfDoencas, dfBackground, [0.1, 0.5], hierarquias)

In [17]:
final

array([['*/*/2000', 'Female', '*, New York', 'Virose'],
       ['*/*/1999', 'Female', '*, Kansas', 'Zika'],
       ['*/*/1999', 'Female', '*, New York', 'Virose'],
       ...,
       ['*/*/2000', 'Male', '*, New York', 'Gripe'],
       ['*/*/2000', 'Male', '*, New York', 'Virose'],
       ['*/*/2000', 'Male', '*, Kansas', 'Gripe']], dtype=object)